In [1]:
import pytorch_lightning as pl
import pytorch_lightning.callbacks as pl_callbacks
import torch
import eq

## Loading the catalog

By default, all catalogs are saved to the following directory

In [2]:
print(eq.data.default_catalogs_dir)

/home/gcl/RA/zekai.wang/recast/data


We load the catalog constructed by White et al. The catalog records earthquakes between `2008-01-01` and `2021-01-01`.
The following code automatically downloads and preprocesses the catalog, or loads it from the data directory, if it's already there.

In [3]:
catalog = eq.catalogs.ANSS_MultiCatalog(num_sequences=10000,
    t_end_days=4*365,
    mag_completeness=4.5,
    minimum_mainshock_mag=6.0,)

Loading existing catalog from /home/gcl/RA/zekai.wang/recast/data/ANSS_MultiCatalog.


/home/gcl/RA/zekai.wang/recast/eq/data/sequence.py:213: UserWarning: Found 1 zero inter-event times in the sequence. This violates fundamental assumptions of TPP models and may lead to incorrect log-likelihood values.
  warnings.warn(


Other available catalogs:
- `eq.catalogs.SCEDC`
- `eq.catalogs.QTMSanJacinto`
- `eq.catalogs.QTMSaltonSea`

### Representing the data
We can represent all data stored in the catalog as an object of type `eq.data.Sequence`.

In [4]:
catalog.full_sequence

AttributeError: 'ANSS_MultiCatalog' object has no attribute 'full_sequence'

An `eq.data.Sequence` object contains the following attributes:
- `t_start` and `t_end`: Start and end of the time interval containing the events (in days).
- `arrival_times`: Arrival times of each event (in days), shape `[num_events]`
- `inter_times`: Inter-event times, shape `[num_events + 1]` since this also includes the last survival time from `arrival_times[-1]` until `t_end`.
- `mag`: Magnitude of each earthquake, shape `[num_events]`.
- `t_nll_start`: This attributed allows to compute the negative log-likelihood (NLL) only for the observations in the interval `[t_nll_start, t_end]`, while conditioning the model on past events in `[t_start, t_nll_start]`.

In [ ]:
eq.visualization.visualize_sequence(catalog.full_sequence)

AttributeError: 'ANSS_MultiCatalog' object has no attribute 'full_sequence'

### Train / val / test split

To compare different models, we split the full event sequence into 3 components:
- `train`: used for training the models, includes events between `2008-01-01` and `2014-01-01`.
- `val`: used for early stopping and hyperparameter selection, includes events between `2014-01-01` and `2017-01-01`.
- `test`: used for final evaluation, includes events between `2017-01-01` and `2021-01-01`.

We already provided the start of the val and test intervals when loading the catalog, so now the train, val and test sequences are stored in `catalog.train`, `catalog.val` and `catalog.test`, respectively.

In our case the catalog consists of a single event sequence but, in principle, there could be multiple event sequences stored in `catalog.train`, `catalog.val` and `catalog.test`.

In [ ]:
print("Train sequence:")
print(catalog.train[0])
print("\nValidation sequence:")
print(catalog.val[0])
print("\nTest sequence:")
print(catalog.test[0])

Train sequence:
Sequence(
  inter_times: [2261],
  arrival_times: [2260],
  t_start: 0.0,
  t_end: 1460.0,
  t_nll_start: 0.0,
  mag: [2260]
)

Validation sequence:
Sequence(
  inter_times: [791],
  arrival_times: [790],
  t_start: 0.0,
  t_end: 1460.0,
  t_nll_start: 0.0,
  mag: [790]
)

Test sequence:
Sequence(
  inter_times: [438],
  arrival_times: [437],
  t_start: 0.0,
  t_end: 1460.0,
  t_nll_start: 0.0,
  mag: [437]
)


Note that, for example, the `test` sequence contains all events in the catalog, including the training set.

However, since we specified `t_nll_start`, the model will know to only compute the NLL on the interval `[t_nll_start, t_end]` (shown in orange in the figure below), while also conditioning on the events from `[0, t_nll_start]`.

<Axes: xlabel='Time (days)', ylabel='Magnitude'>

In [5]:
eq.visualization.visualize_sequence(catalog.test[0], show_nll=True)

In [6]:
len(catalog.test[0])

437

## Training the model

We will train the model using the high-level API provided by `pytorch_lightning`.

For this, we first need to create `torch.utils.data.DataLoader` objects that will feed the data into the model.

In [7]:
dl_train = catalog.train.get_dataloader(batch_size=1)
dl_val = catalog.val.get_dataloader(batch_size=1)
dl_test = catalog.test.get_dataloader(batch_size=1)

Now, we initialize the model.

In [11]:
model = eq.models.RecurrentTPP_Attention(learning_rate=5e-2, tau_mean=3)

`pytorch_lightning.Trainer` takes care of training the model, early stopping and saving the best weights.

In this toy example, we set `max_epochs=200` and `patience=20` to get to the results more quickly.

In the experiments in the paper, we trained the models for longer with `max_epochs=1500` and `patience=200`.

In [12]:
# ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="val_loss")

# EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="val_loss", patience=20, min_delta=1e-3)

# num_gpus = int(torch.cuda.is_available())  # equals to 1 if GPU is available, 0 otherwise
trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=200, callbacks=[checkpoint, early_stopping])
# I think I can train on multiple gpus, but for now I just focus on training on a single gpu
# It seems that there is dataracing when I try to use multiple gpus

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(model, dl_train, dl_val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name           | Type               | Params
------------------------------------------------------
0 | hypernet_time  | Linear             | 3.2 K 
1 | hypernet_mag   | Linear             | 33    
2 | rnn            | GRU                | 3.5 K 
3 | dropout        | Dropout            | 0     
4 | multihead_attn | MultiheadAttention | 4.2 K 
------------------------------------------------------
10.9 K    Trainable params
0         Non-trainable params
10.9 K    Total params
0.044     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/gcl/RA/zekai.wang/miniconda3/envs/eq/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


AttributeError: 'Batch' has no attribute 'mag_bounds'

In [36]:
trainer

## Evaluation

Load the model with the best validation loss

In [ ]:
model.load_from_checkpoint(checkpoint.best_model_path)

After training the model, we compute the loss on the negative log-likelihood (NLL) loss on the test set.

In [ ]:
trainer.test(model, dl_test)